In [52]:
import numpy as np
import pandas as pd
from survONS import SurvONS
from utils import date_discretization
import matplotlib.pyplot as plt

df = pd.read_csv("../../../ventasNIC_censura.csv")
df.drop(labels="Unnamed: 0", axis=1, inplace=True)
df.head()

,hora,pais_compra,rut_hash,nac_titular,empresa,dominio,pais,region,moneda,monto,...,anho,mes,dia,tiempo_tarifa,anho_renov,mes_renov,dia_renov,tiempo,time,cens
0,22,cl,-8746099959568201971,0,2,lostroncosdelacruz.cl,cl,5,CLP,49950.0,...,2024,9,4,5,2029,9,4,129,0.0,False
1,16,cl,-1772222548298478989,1964,1,almondale.cl,cl,8,CLP,99900.0,...,2024,9,4,10,2034,9,4,129,0.0,False
2,10,cl,8961343460819268678,1984,1,kul.cl,cl,13,CLP,9990.0,...,2024,9,4,1,2025,9,4,129,0.0,False
3,10,cl,8510801244394156401,1979,1,ailinco.cl,cl,10,CLP,19980.0,...,2024,9,4,2,2026,9,4,129,0.0,False
4,10,cl,-2232705689846604328,0,2,coneypark.cl,cl,13,CLP,19980.0,...,2024,9,4,2,2026,9,4,129,0.0,False


In [53]:
i_dates_df = df[["anho", "mes", "dia"]][0:150000].rename(columns={"anho": "year", "mes": "month", "dia": "day"})
f_dates_df = df[["anho_renov", "mes_renov", "dia_renov"]][0:150000].rename(columns={"anho_renov": "year", "mes_renov": "month", "dia_renov": "day"})
dates_df = pd.concat([i_dates_df, f_dates_df])
dates=date_discretization(pd.to_datetime(dates_df[["year", "month", "day"]]).to_numpy())
print(dates)



[156 156 156 ... 471 314 157]


In [54]:
df = df[0:150000]
#print(dates[0:150000])
df.insert(0, "initial_time", dates[0:150000])
df.insert(19,"final_time", dates[150000:300000])
df.drop(labels=["anho","mes","dia","anho_renov","mes_renov","dia_renov", "pais_compra"], axis=1, inplace=True)
df.head()

,initial_time,hora,rut_hash,nac_titular,empresa,dominio,pais,region,moneda,monto,tipo_tarifa,tiempo_tarifa,final_time,tiempo,time,cens
0,156,22,-8746099959568201971,0,2,lostroncosdelacruz.cl,cl,5,CLP,49950.0,15,5,941,129,0.0,False
1,156,16,-1772222548298478989,1964,1,almondale.cl,cl,8,CLP,99900.0,20,10,1311,129,0.0,False
2,156,10,8961343460819268678,1984,1,kul.cl,cl,13,CLP,9990.0,11,1,313,129,0.0,False
3,156,10,8510801244394156401,1979,1,ailinco.cl,cl,10,CLP,19980.0,2,2,470,129,0.0,False
4,156,10,-2232705689846604328,0,2,coneypark.cl,cl,13,CLP,19980.0,32,2,470,129,0.0,False


In [55]:
df["pais"] = pd.Categorical(df["pais"]).codes
df["moneda"] = pd.Categorical(df["moneda"]).codes



In [56]:
df.drop(labels=["dominio", "tiempo", "time"], axis=1, inplace=True)

In [57]:
x = df.drop(labels=["final_time", "cens"], axis=1)

surv = SurvONS()
surv.train(x, df["initial_time"].to_numpy(), df["final_time"].to_numpy(), df["cens"].to_numpy())

Valor de D muy pequeño. Probando con D=2


d:\dge20\Documentos\RamosUniversidad\10moSemestre\Software\venvs\pysurvons\lib\site-packages\cvxpy\problems\problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Generalized projection error: optimization is not optimal and ends with status optimal_inaccurate None
Entrenamiento exitoso, para optimizar el modelo utilizar iterative_train


In [64]:
surv.predict(x.to_numpy()[1000], 1000)

np.float64(nan)